<a href="https://colab.research.google.com/github/DowookKim/Deep_learing/blob/master/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-amns1hh3
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-amns1hh3
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286920 sha256=e03bcb0ce90a1a36bc5bb48aa5ac54d20ad657d2698aab0bfc2b53adae53d54f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_m8y_6n/wheels/76/1f/66/ecf5b2f136847d084f6e9af3c732f2899ee9f9b229a45aa8b8
Successfully built pykospacing


In [ ]:
!pip install soynlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [ ]:
import tensorflow as tf
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)
#score = self.V(tf.nn.tanh(
#    self.W1(values) + self.W2(hidden_with_time_axis)))

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # 디코더의 shape이 이런 이유는 시퀀스에서 단어 하나에 대해서만
    # batch size와 hidden size가 있기 때문이다.
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    #여기서 **1이 의미하는 것은 “시퀀스 길이 1”**입니다.
    # 즉, decoder의 현재 시점은 하나뿐이라는 뜻이에요.

    # encoder의 shape은 (batch_size, max_length, hidden_size)
    # 문장 4개가 있다고 하면 batch_size = 4, 각각의 문장이 10개의 단어를 가지면
    # max_length = 10, 각각의 단어에 대해서 256 차원이면 hidden size = 256이다.
    # 그래서 encoder와 decoder의 차원을 맞춰주기 위해서
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1) #가중합을 위한 함수

    return context_vector, attention_weights


In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)


(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [ ]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태


In [ ]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'bahdanau_attention_3' (of type BahdanauAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [ ]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)


Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 983s 10s/step - accuracy: 0.6217 - loss: 0.6161 - val_accuracy: 0.8705 - val_loss: 0.3131
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 1003s 10s/step - accuracy: 0.9014 - loss: 0.2932 - val_accuracy: 0.8840 - val_loss: 0.2812
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 932s 10s/step - accuracy: 0.9318 - loss: 0.2262 - val_accuracy: 0.8842 - val_loss: 0.2773


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))


 12/782 ━━━━━━━━━━━━━━━━━━━━ 4:38 362ms/step - accuracy: 0.8808 - loss: 0.2484

KeyboardInterrupt: 